In [ ]:
#Upgrade Setuptools if Needed
!pip install --upgrade setuptools wheel

#Install prerequisite packages
!pip install python-dotenv==1.0.0

!pip install llama-index==0.10.59
!pip install llama-index-llms-openai==0.1.27
!pip install llama-index-embeddings-openai==0.1.11
!pip install llama-index-llms-azure-openai==0.1.10
!pip install llama-index-embeddings-azure-openai==0.1.11


In [5]:

#Setup Azure Open AI connection
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from llama_index.core import Settings
import os
import nest_asyncio

nest_asyncio.apply()

#API info. Replace with your own keys and end points
api_key = "e638bb3ebcb84b79aa5b6f93d6e6503a"
azure_endpoint = "https://agentic-ai-course-account.openai.azure.com/"

#api_key = "provide your key"
#azure_endpoint = "provide your end point"
api_version = "2024-05-01-preview" #Update version if required

#Setup the LLM
Settings.llm=AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="agentai-gpt35",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

#Setup the embedding model RAG
Settings.embed_model= AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="agentai-embedding",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)



In [6]:
#Create indexes for vector search
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex

splitter=SentenceSplitter(chunk_size=1024)

#-------------------------------------------------------------------
#Setup IPC document index
#-------------------------------------------------------------------
IPC_documents=SimpleDirectoryReader(
    input_files=["NyaySanhita2023.pdf"])\
            .load_data()
#Read documents into nodes
IPC_nodes=splitter.get_nodes_from_documents(IPC_documents)
#Create a vector Store
IPC_index=VectorStoreIndex(IPC_nodes)
#Create a query engine
IPC_query_engine = IPC_index.as_query_engine()





In [7]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


#Create a query engine Tool for NLP
IPC_tool = QueryEngineTool.from_defaults(
    query_engine=IPC_query_engine,
    name="Indian Penal Code",
    description=(
        "Contains information the law and order in India and the probable punishments"
    ),
)

#Create a Router Agent. Provide the Tools to the Agent
router_agent=RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
       IPC_tool,
    ],
    verbose=True
)

In [8]:
response = router_agent.query("What is the punishment for murder in India?")
print("\nResponse: ",str(response))

Selecting query engine 0: Choice 1 contains information about the law and order in India, which may include the probable punishments for crimes like murder..

Response:  The punishment for murder in India is death or imprisonment for life, along with a possible fine.


In [9]:
response = router_agent.query("What is the punishment for attempt to murder in India?")
print("\nResponse: ",str(response))

Selecting query engine 0: Choice 1 contains information about the law and order in India, which includes probable punishments. Therefore, it is the most relevant choice to answer the question about the punishment for attempt to murder in India..

Response:  The punishment for attempt to murder in India is imprisonment of either description for a term which may extend to ten years, and the offender may also be liable to a fine.


In [10]:
response = router_agent.query("What is section 302 as per Indian laws?")
print("\nResponse: ",str(response))

Selecting query engine 0: Choice 1 contains information about the law and order in India, which is relevant to understanding section 302 as per Indian laws..

Response:  Section 302 of the Indian Penal Code deals with the offense of murder. It states that whoever commits murder shall be punished with the death penalty or imprisonment for life, and shall also be liable to pay a fine.


In [11]:
response = router_agent.query("What is punishment for drunk driving in India?")
print("\nResponse: ",str(response))

Selecting query engine 0: Choice 1 contains information about the law and order in India, which would likely include information about the punishment for drunk driving..

Response:  The punishment for drunk driving in India may include a fine and/or imprisonment. The specific penalties may vary depending on the circumstances and the applicable laws in the country.
